# Examples
This notebook shows some examples of how to use the recommendation server.

## Setup

In [1]:
import requests
import json

Let's define some variable we are going to use in the examples

In [2]:
rec_server_base_url = "http://127.0.0.1:5000/"
rec_algos = ["popular"]
pred_algos = ["bias","itemitem","useruser","biasedmf","implicitmf","funksvd","tf_bpr"]
user_id = 1
nr_recs = 10
items = "10,20,30,40,50"

## Recommendations

In [3]:
for algo in rec_algos:
    url = f'{rec_server_base_url}/algorithms/{algo}/recommendations?user_id={user_id}&num_recs={nr_recs}'
    r = requests.get(url)
    print(json.dumps(r.json(), indent=2))

{
  "recommendations": [
    {
      "item": 356.0,
      "score": 341.0
    },
    {
      "item": 318.0,
      "score": 311.0
    },
    {
      "item": 593.0,
      "score": 304.0
    },
    {
      "item": 260.0,
      "score": 291.0
    },
    {
      "item": 480.0,
      "score": 274.0
    },
    {
      "item": 2571.0,
      "score": 259.0
    },
    {
      "item": 1.0,
      "score": 247.0
    },
    {
      "item": 527.0,
      "score": 244.0
    },
    {
      "item": 589.0,
      "score": 237.0
    },
    {
      "item": 1196.0,
      "score": 234.0
    }
  ]
}


## Predictions

In [4]:
for algo in pred_algos:
    print(f"Algorithm: {algo}")
    url = f'{rec_server_base_url}/algorithms/{algo}/predictions?user_id={user_id}&items={items}'
    r = requests.get(url)
    print(json.dumps(r.json(), indent=2))
    print()    
    print('-----------------------------')
    print()

Algorithm: bias
{
  "predictions": [
    {
      "item": 10,
      "score": 3.4674757437886936
    },
    {
      "item": 20,
      "score": 2.5551176101190847
    },
    {
      "item": 30,
      "score": 4.066656071657546
    },
    {
      "item": 40,
      "score": 3.933322738324213
    },
    {
      "item": 50,
      "score": 4.387302837826703
    }
  ]
}

-----------------------------

Algorithm: itemitem
{
  "predictions": [
    {
      "item": 10,
      "score": 3.231073190470284
    },
    {
      "item": 20,
      "score": 2.9305180210428925
    },
    {
      "item": 30,
      "score": 4.073950231233046
    },
    {
      "item": 40,
      "score": 3.986381186580402
    },
    {
      "item": 50,
      "score": 4.580127129467687
    }
  ]
}

-----------------------------

Algorithm: useruser
{
  "predictions": [
    {
      "item": 10,
      "score": 3.4913655291780747
    },
    {
      "item": 20,
      "score": 3.516813277855599
    },
    {
      "item": 30,
      "scor

## Get model information

In [5]:
for algo in pred_algos:
    print(f"Algorithm: {algo}")
    url = f'{rec_server_base_url}/algorithms/{algo}/info'
    r = requests.get(url)
    print(json.dumps(r.json(), indent=2))
    print()    
    print('-----------------------------')
    print()    

Algorithm: bias
{
  "model": {
    "creation_date": "2020-10-19 23:42:14",
    "size": 739.388,
    "updated_date": "2020-10-19 23:42:14"
  }
}

-----------------------------

Algorithm: itemitem
{
  "model": {
    "creation_date": "2020-10-21 16:47:08",
    "size": 212330.473,
    "updated_date": "2020-10-21 16:47:08"
  }
}

-----------------------------

Algorithm: useruser
{
  "model": {
    "creation_date": "2020-10-15 21:16:28",
    "size": 688.368,
    "updated_date": "2020-08-21 18:33:27"
  }
}

-----------------------------

Algorithm: biasedmf
{
  "model": {
    "creation_date": "2020-10-15 21:16:28",
    "size": 3973.614,
    "updated_date": "2020-10-08 21:10:51"
  }
}

-----------------------------

Algorithm: implicitmf
{
  "model": {
    "creation_date": "2020-10-15 21:16:28",
    "size": 1507.69,
    "updated_date": "2020-08-21 18:33:45"
  }
}

-----------------------------

Algorithm: funksvd
{
  "model": {
    "creation_date": "2020-10-15 21:16:28",
    "size": 1522.498

## Upload a model file

In [6]:
current_algo = rec_algos[0]
url = f'{rec_server_base_url}/algorithms/{current_algo}/modelfile'
model_name = current_algo + ".bpk"
files = {
    'file': open(model_name, 'rb')
}
response = requests.put(url, files=files)
print(json.dumps(r.json(), indent=2))

{
  "model": {
    "creation_date": "2020-10-27 17:25:43",
    "size": 19515.322,
    "updated_date": "2020-10-27 17:25:43"
  }
}


## Extending the recommendation server
An example of extending the recommendation server:

Create a new custom endpoint where we return the worst predictions for the algo sent in the request.

We created a new method called `get_worst_predictions_from_model` in model_manager.py and another endpoint called `get_worst_predictions` with the python decorator: 
    @models.model_method("worst_predictions", Predictor, models.get_worst_predictions_from_model, models.get_preds_params)

And that's it, we have a new endpoint that returns the worst predictions for an algorithm. The code is shown below:

In [7]:
def get_worst_predictions_from_model(self, model, *args):
    user, items = None, None
    try:
        user, items, ratings = args[0][0], args[0][1], args[0][2]
        results = []
        df_preds = model.predict_for_user(user, items, ratings)
        for index, value in df_preds.iteritems():
            if not math.isnan(value):
                results.append({'item': index, 'score': value})
        results = sorted(results, key = lambda i: i['score'])
        return results
    except:
        logging.error(f"Unexpected preds error for user: {user}, with items: {items}. Error: {sys.exc_info()[0]}")
        raise

In [8]:
def get_worst_predictions(results):
    """
    Get worst predictions using the algorithm, user id and items sent.
    Args:
        algo: algorithm to be used.
        user_id: user id to get predictions for.
        items: items to get predictions for.
    Returns:
        A list of predictions with items and scores.
    """
    return jsonify({"predictions": results})

Let's test the new endpoint.

In [9]:
for algo in pred_algos:
    print(f"Algorithm: {algo}")
    url = f'{rec_server_base_url}/algorithms/{algo}/worst_predictions?user_id={user_id}&items={items}'
    r = requests.get(url)
    print(json.dumps(r.json(), indent=2))
    print()    
    print('-----------------------------')
    print()

Algorithm: bias
{
  "predictions": [
    {
      "item": 20,
      "score": 2.5551176101190847
    },
    {
      "item": 10,
      "score": 3.4674757437886936
    },
    {
      "item": 40,
      "score": 3.933322738324213
    },
    {
      "item": 30,
      "score": 4.066656071657546
    },
    {
      "item": 50,
      "score": 4.387302837826703
    }
  ]
}

-----------------------------

Algorithm: itemitem
{
  "predictions": [
    {
      "item": 20,
      "score": 2.9305180210428925
    },
    {
      "item": 10,
      "score": 3.231073190470284
    },
    {
      "item": 40,
      "score": 3.986381186580402
    },
    {
      "item": 30,
      "score": 4.073950231233046
    },
    {
      "item": 50,
      "score": 4.580127129467687
    }
  ]
}

-----------------------------

Algorithm: useruser
{
  "predictions": [
    {
      "item": 10,
      "score": 3.4913655291780747
    },
    {
      "item": 20,
      "score": 3.516813277855599
    },
    {
      "item": 30,
      "scor